In [76]:

%pip install pandas

import requests
import logging
import pandas as pd
import sys
import os
import json

# Add the parent directory to sys.path so 'Data' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Call the API endpoint
response = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
data = response.json()

# Get the count of events
event_count = len(data.get("events", []))

# Log the count of events
print(f"Count of events: {event_count}")

# Pull the CSV file into memory as a pandas DataFrame
file_path = "/Users/owen/src/Personal/fpl-team-picker/Data/raw/parsed_gw.csv"
xpData = pd.read_csv(file_path)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Count of events: 38


In [77]:
%pip install scipy
from Data.utils.lookup_xp import lookup_fpl_data
from scipy import stats

def get_scatter_data_for_player(element_type, previous_fixtures, teams):
  """
  Given a player dict (from players.json) and a list of teams (from teams.json),
  returns a list of (opponent_strength, points_scored) tuples for each previous fixture.
  The opponent_strength is chosen based on the player's element_type and was_home status:
    - element_type 1 or 2 (GK/MID): use opponent's attack strength
    - element_type 3 or 4 (DEF/FWD): use opponent's defence strength
    - was_home True: use opponent's *_away strength
    - was_home False: use opponent's *_home strength
  """
  # Build a mapping from team id to team dict
  team_map = {team.get('id'): team for team in teams}

  scatter_data = []
  for fixture in previous_fixtures:
    opponent_team_id = fixture['opponent_team']
    opponent_team = team_map.get(opponent_team_id)
    if not opponent_team:
      continue

    was_home = fixture['was_home']

    if element_type in (1, 2):  # GK or DEF
      if was_home:
        opponent_strength = opponent_team.get('strength_attack_away')
      else:
        opponent_strength = opponent_team.get('strength_attack_home')
    elif element_type in (3, 4):  # MID or FWD
      if was_home:
        opponent_strength = opponent_team.get('strength_defence_away')
      else:
        opponent_strength = opponent_team.get('strength_defence_home')
    else:
      opponent_strength = None

    points_scored = fixture['total_points']
    if fixture.get('minutes', 0) == 0:
      continue
    if opponent_strength is not None:
      scatter_data.append((opponent_strength, points_scored))

  if len(scatter_data) >= 2:
    x, y = zip(*scatter_data)
    slope, intercept, *_ = stats.linregress(x, y)
  else:
    slope, intercept = None, None

  return slope, intercept, scatter_data

players = data.get("elements", [])
teams = data.get("teams", [])

players_data = []

# Iterate through each player and fetch their fixture data
for index, player in enumerate(players):
  if player['element_type'] < 5:
    players_data.append(player)
  player_id = player['id']
  logging.info(f"Processing player {index + 1}/{len(players)}: {player['web_name']} (ID: {player_id})")

  # Call the API to get the player's fixtures
  url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
  response = requests.get(url)

  if response.status_code == 200:
    player_data = response.json()
    player['upcoming_fixtures'] = player_data.get('fixtures', [])
    player['previous_fixtures'] = player_data.get('history', [])

    slope, intercept, scatter_data = get_scatter_data_for_player(player.get('element_type'), player.get('previous_fixtures', []), teams)
    player['opponent_strengths'] = {
      "slope": slope,
      "intercept": intercept,
      "data": scatter_data
    }
    for index, gw in enumerate(player['previous_fixtures']):
      found_row = lookup_fpl_data(player_id, gw['round'], xpData)
      if found_row is not None:
         logging.info(f"Found row for player {player['web_name']} in gameweek {gw['round']}, with xP: {found_row.xP}")
         gw['xp'] = found_row['xP']
       
  else:
    logging.warning(f"Failed to fetch data for player ID {player_id}. Status code: {response.status_code}")

# Save the extracted data to a JSON file
with open("database/players.json", "w") as file:
  json.dump(players_data, file, indent=4)

print("Players saved.")


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


2025-05-27 20:34:29,006 - INFO - Processing player 1/804: Fábio Vieira (ID: 1)


Note: you may need to restart the kernel to use updated packages.


2025-05-27 20:34:29,133 - INFO - Found row for player Fábio Vieira in gameweek 1, with xP: 2.3
2025-05-27 20:34:29,136 - INFO - Found row for player Fábio Vieira in gameweek 2, with xP: 0.8
2025-05-27 20:34:29,137 - INFO - Found row for player Fábio Vieira in gameweek 3, with xP: 0.0
2025-05-27 20:34:29,139 - INFO - Found row for player Fábio Vieira in gameweek 4, with xP: 0.0
2025-05-27 20:34:29,140 - INFO - Found row for player Fábio Vieira in gameweek 5, with xP: 0.0
2025-05-27 20:34:29,141 - INFO - Found row for player Fábio Vieira in gameweek 6, with xP: 0.0
2025-05-27 20:34:29,142 - INFO - Found row for player Fábio Vieira in gameweek 7, with xP: 0.0
2025-05-27 20:34:29,143 - INFO - Found row for player Fábio Vieira in gameweek 8, with xP: 0.0
2025-05-27 20:34:29,144 - INFO - Found row for player Fábio Vieira in gameweek 9, with xP: 0.0
2025-05-27 20:34:29,145 - INFO - Found row for player Fábio Vieira in gameweek 10, with xP: 0.0
2025-05-27 20:34:29,146 - INFO - Found row for pl

Players saved.


In [78]:
import json

# Save the teams data to a JSON file
with open("database/teams.json", "w") as file:
  json.dump(teams, file, indent=4)

print("Teams saved to database/teams.json.")

Teams saved to database/teams.json.
